# Publish Data

Publish the data structure to ArcGIS Online

#### Import Statements

In [1]:
# import the arcpy libary
import arcpy
arcpy.env.overwriteOutput = True

import arcgis
gis = arcgis.gis.GIS(profile="ag")

# import other modules
import json
import copy
import os
from getpass import getpass

password = getpass('Password: ')
arcpy.SignInToPortal('https://www.arcgis.com', 'ag_svcs', password)

Password: ········


{'token': 'KmgfUKIredZ68J3mRatXG-HHgnkS5xHZMxm-Kr0lejgYTMfDjP6BCzEUGBCZwZVQaGJMV8VpAU-YvR2SzokXtZ1S7GuJtdzDfO6ry1W5Ux3whkQPx6QCEhLHHtuZ1T_kkTIFlW9HCFR0LapdmG3DeIPKBF1L4wh4xX61ek_WWTAEu4Kj2W2md9CQ93Rifi2VK1QQleeEuF0tK4zjmCRB3-4-jE9eKyfR8Dj8oK6zDLQ.',
 'referer': 'http://www.esri.com/AGO/5224E2FE-95AE-4A6F-BDE4-1B94680D2040',
 'expires': 1642688128}

#### Set output path

In [2]:
basePath = r'C:\Users\rjc.EAGLE\Documents\GitHub\agriculture-data-model'
outputFolders = os.listdir('{0}\\output'.format(basePath))

n = 0
for folder in outputFolders:
    print('{0} - {1}'.format(n, folder))
    n+=1
    
selectedFolder = int(input('Select a output folder: '))
outputPath = '{0}\\output\\{1}'.format(basePath, outputFolders[selectedFolder])
print('Output path: {0}'.format(outputPath))

0 - example-farm
1 - test
Select a output folder: 0
Output path: C:\Users\rjc.EAGLE\Documents\GitHub\agriculture-data-model\output\example-farm


#### Set paths and load in Data Model

In [3]:
data = {}
try:    
    data.update(json.load(open('{0}\\service\\{1}.json'.format(outputPath, 'datamodel'))))
except Exception as e:
    print (e)

#### Publish data

In [4]:
aprx = arcpy.mp.ArcGISProject('{0}\\project\\{1}.aprx'.format(basePath, 'publish'))
publishMap = aprx.listMaps('Map')[0]

n = 0
for service in data['services']:
    
    print(service['name'])    
    serviceName = service['name']
    serviceDefinitionDraftPath = '{0}\\service\\{1}.sddraft'.format(outputPath, serviceName)
    serviceDefinitionPath = '{0}\\service\\{1}.sd'.format(outputPath, serviceName)
    
    layers = publishMap.listLayers()
    for layer in layers:
        publishMap.removeLayer(layer)
    
    tables = publishMap.listTables()
    for table in tables:
        publishMap.removeTable(table)
        
    for layer in service['layers'][::-1]:
        path = '{0}\\data\\{1}.gdb\\{2}'.format(outputPath, serviceName, layer['name'])
        publishMap.addDataFromPath(path)
        
    for table in service['tables'][::-1]:
        path = '{0}\\data\\{1}.gdb\\{2}'.format(outputPath, serviceName, table)
        publishMap.addDataFromPath(path)
        
    layers = publishMap.listLayers()
    tables = publishMap.listTables()
    items = layers + tables
    aprx.save()
    
    serviceDefinitionDraft = publishMap.getWebLayerSharingDraft('HOSTING_SERVER', "FEATURE", serviceName, items)
    
    serviceDefinitionDraft.credits = "These are credits"
    serviceDefinitionDraft.description = "This is description"
    serviceDefinitionDraft.summary = "This is summary"
    serviceDefinitionDraft.tags = "tag1, tag2"
    serviceDefinitionDraft.useLimitations = "These are use limitations"
    serviceDefinitionDraft.portalFolder = "Farm Data"
    serviceDefinitionDraft.allowExporting = False
    serviceDefinitionDraft.enableEditing = True
    serviceDefinitionDraft.enableSync = True

    serviceDefinitionDraft.exportToSDDraft(serviceDefinitionDraftPath)
    
    print("  - staging")
    staged = arcpy.StageService_server(serviceDefinitionDraftPath, serviceDefinitionPath)
    print("    + completed")

    # Share to portal
    print("  - uploading")
    uploaded = arcpy.UploadServiceDefinition_server(serviceDefinitionPath, 'HOSTING_SERVER')
    data['services'][n]['id'] = uploaded.getOutput(3)
    n+=1
    print("    + completed")

administrative_information
  - staging
    + completed
  - uploading
    + completed
structures
  - staging
    + completed
  - uploading
    + completed
operations
  - staging
    + completed
  - uploading
    + completed
transport
  - staging
    + completed
  - uploading
    + completed
electric_utilities
  - staging
    + completed
  - uploading
    + completed
water_utilities
  - staging
    + completed
  - uploading
    + completed
effluent_utilities
  - staging
    + completed
  - uploading
    + completed
topographic_features
  - staging
    + completed
  - uploading
    + completed
miscellaneous
  - staging
    + completed
  - uploading
    + completed
water
  - staging
    + completed
  - uploading
    + completed
land
  - staging
    + completed
  - uploading
    + completed
insights
  - staging
    + completed
  - uploading
    + completed
activities
  - staging
    + completed
  - uploading
    + completed
health_and_safety
  - staging
    + completed
  - uploading
    + c

#### Update data model with service id's

In [5]:
with open('{0}\\service\\{1}.json'.format(outputPath, 'datamodel'), 'w') as file:
    file.write(json.dumps(data))